In [12]:
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Parameter as TorchParam
from torch import Tensor
from typing import List, Tuple
from pprint import pprint

In [13]:
path_to_csv = r"C:\20_Develop\10_Programming\60_thermal-nn\data\input\measures_v2.csv"
data = pd.read_csv(path_to_csv)
target_cols = ["pm", "stator_yoke", "stator_tooth", "stator_winding"]

In [14]:
data.columns

Index(['u_q', 'coolant', 'stator_winding', 'u_d', 'stator_tooth',
       'motor_speed', 'i_d', 'i_q', 'pm', 'stator_yoke', 'ambient', 'torque',
       'profile_id'],
      dtype='object')

In [15]:
temperature_cols = target_cols + ["ambient", "coolant"]
test_profiles = [60, 62, 74]
train_profiles = [p for p in data.profile_id.unique() if p not in test_profiles]
profile_sizes = data.groupby("profile_id").agg("size")

In [16]:
non_temperature_cols = [c for c in data if c not in temperature_cols + ["profile_id"]]
non_temperature_cols

['u_q', 'u_d', 'motor_speed', 'i_d', 'i_q', 'torque']

In [17]:
data.loc[:, temperature_cols].describe()

,pm,stator_yoke,stator_tooth,stator_winding,ambient,coolant
count,1.330816e+06,1.330816e+06,1.330816e+06,1.330816e+06,1.330816e+06,1.330816e+06
mean,5.850678e+01,4.818796e+01,5.687858e+01,6.634275e+01,2.456526e+01,3.622999e+01
std,1.900150e+01,1.999100e+01,2.295223e+01,2.867206e+01,1.929522e+00,2.178615e+01
min,2.085696e+01,1.807669e+01,1.813398e+01,1.858582e+01,8.783478e+00,1.062375e+01
25%,4.315158e+01,3.199033e+01,3.841601e+01,4.278796e+01,2.318480e+01,1.869814e+01
50%,6.026629e+01,4.562551e+01,5.603635e+01,6.511013e+01,2.479733e+01,2.690014e+01
75%,7.200837e+01,6.146083e+01,7.558668e+01,8.814114e+01,2.621702e+01,4.985749e+01
max,1.136066e+02,1.011480e+02,1.119464e+02,1.413629e+02,3.071420e+01,1.015985e+02


In [18]:
data.loc[:, temperature_cols] /= 200  # deg C
data.loc[:, temperature_cols].describe()

,pm,stator_yoke,stator_tooth,stator_winding,ambient,coolant
count,1.330816e+06,1.330816e+06,1.330816e+06,1.330816e+06,1.330816e+06,1.330816e+06
mean,2.925339e-01,2.409398e-01,2.843929e-01,3.317137e-01,1.228263e-01,1.811500e-01
std,9.500749e-02,9.995502e-02,1.147612e-01,1.433603e-01,9.647609e-03,1.089307e-01
min,1.042848e-01,9.038344e-02,9.066988e-02,9.292908e-02,4.391739e-02,5.311876e-02
25%,2.157579e-01,1.599517e-01,1.920801e-01,2.139398e-01,1.159240e-01,9.349068e-02
50%,3.013314e-01,2.281275e-01,2.801817e-01,3.255506e-01,1.239866e-01,1.345007e-01
75%,3.600419e-01,3.073041e-01,3.779334e-01,4.407057e-01,1.310851e-01,2.492874e-01
max,5.680331e-01,5.057398e-01,5.597321e-01,7.068144e-01,1.535710e-01,5.079926e-01


In [19]:
data.loc[:, non_temperature_cols] /= data.loc[:, non_temperature_cols].abs().max(axis=0)
data.loc[:, non_temperature_cols].describe()

,u_q,u_d,motor_speed,i_d,i_q,torque
count,1.330816e+06,1.330816e+06,1.330816e+06,1.330816e+06,1.330816e+06,1.330816e+06
mean,4.079993e-01,-1.910874e-01,3.670125e-01,-2.471795e-01,1.240033e-01,1.191776e-01
std,3.320372e-01,4.796759e-01,3.099431e-01,2.335697e-01,3.055336e-01,2.955328e-01
min,-1.901045e-01,-1.000000e+00,-4.592474e-02,-1.000000e+00,-9.725527e-01,-9.442961e-01
25%,9.072608e-02,-5.982716e-01,5.285164e-02,-4.151243e-01,3.632199e-03,-5.265269e-04
50%,3.678539e-01,-5.648697e-02,3.333286e-01,-1.837881e-01,5.228239e-02,4.160964e-02
75%,6.767621e-01,1.117818e-02,6.267716e-01,-1.071816e-02,3.334754e-01,3.509394e-01
max,1.000000e+00,9.995391e-01,1.000000e+00,1.866763e-04,1.000000e+00,1.000000e+00


In [20]:
# extra feats (FE)
if {"i_d", "i_q", "u_d", "u_q"}.issubset(set(data.columns.tolist())):
    extra_feats = {
        "i_s": lambda x: np.sqrt((x["i_d"] ** 2 + x["i_q"] ** 2)),
        "u_s": lambda x: np.sqrt((x["u_d"] ** 2 + x["u_q"] ** 2)),
    }

In [21]:
data = data.assign(**extra_feats)
input_cols = [c for c in data.columns if c not in target_cols]
input_cols

['u_q',
 'coolant',
 'u_d',
 'motor_speed',
 'i_d',
 'i_q',
 'ambient',
 'torque',
 'profile_id',
 'i_s',
 'u_s']

In [22]:
device = torch.device("cpu")

In [23]:
# Rearrange features
input_cols = [c for c in data.columns if c not in target_cols + ["profile_id"]]
data = data.loc[:, input_cols + ["profile_id"] + target_cols]
data.head()

,u_q,coolant,u_d,motor_speed,i_d,i_q,ambient,torque,i_s,u_s,profile_id,pm,stator_yoke,stator_tooth,stator_winding
0,-0.003388,0.094026,-0.002661,4.775934e-07,1.589597e-05,0.000001,0.099253,0.000717,0.000016,0.004308,17,0.122771,0.091583,0.091466,0.095433
1,-0.002448,0.094093,-0.002325,4.279684e-08,2.179369e-06,-0.000003,0.099253,0.000940,0.000003,0.003376,17,0.122690,0.091575,0.091474,0.095462
2,-0.003314,0.094144,-0.002832,3.924942e-07,4.638742e-06,0.000001,0.099253,0.000677,0.000005,0.004359,17,0.122723,0.091632,0.091470,0.095447
3,-0.002458,0.094178,-0.002404,1.017442e-06,9.202875e-08,0.000007,0.099253,0.000913,0.000007,0.003438,17,0.122770,0.091654,0.091463,0.095415
4,-0.003541,0.094285,-0.002526,5.221358e-07,-2.313523e-04,0.000123,0.099253,0.000798,0.000262,0.004350,17,0.122827,0.091633,0.091457,0.095413


In [24]:
profiles_list = train_profiles
tensor = np.full(
    (profile_sizes[profiles_list].max(), len(profiles_list), data.shape[1] - 1),
    np.nan,
)
tensor.shape


(43971, 66, 14)

In [25]:
def generate_tensor(profiles_list):
    """Returns profiles of the data set in a coherent 3D tensor with
    time-major shape (T, B, F) where
    T : Maximum profile length
    B : Batch size = Amount of profiles
    F : Amount of input features.

    Also returns a likewise-shaped sample_weights tensor, which zeros out post-padded zeros for use
    in the cost function (i.e., it acts as masking tensor)"""
    tensor = np.full(
        (profile_sizes[profiles_list].max(), len(profiles_list), data.shape[1] - 1),
        np.nan,
    )
    for i, (pid, df) in enumerate(
        data.loc[data.profile_id.isin(profiles_list), :].groupby("profile_id")
    ):
        assert pid in profiles_list, f"PID is not in {profiles_list}!"
        tensor[: len(df), i, :] = df.drop(columns="profile_id").to_numpy()
    sample_weights = 1 - np.isnan(tensor[:, :, 0])
    tensor = np.nan_to_num(tensor).astype(np.float32)
    tensor = torch.from_numpy(tensor).to(device)
    sample_weights = torch.from_numpy(sample_weights).to(device)
    return tensor, sample_weights

train_tensor, train_sample_weights = generate_tensor(train_profiles)
test_tensor, test_sample_weights = generate_tensor(test_profiles)

In [26]:
train_tensor.shape, test_tensor.shape

(torch.Size([43971, 66, 14]), torch.Size([25600, 3, 14]))

In [27]:
torch.save(train_tensor, "train_tensor.pt")
torch.save(test_tensor, "test_tensor.pt")
torch.save(train_sample_weights, "train_sample_weights.pt") 
torch.save(test_sample_weights, "test_sample_weights.pt")

In [28]:
train_sample_weights

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [29]:
input_cols

['u_q',
 'coolant',
 'u_d',
 'motor_speed',
 'i_d',
 'i_q',
 'ambient',
 'torque',
 'i_s',
 'u_s']

In [30]:
temperature_cols

['pm', 'stator_yoke', 'stator_tooth', 'stator_winding', 'ambient', 'coolant']

In [31]:
target_cols

['pm', 'stator_yoke', 'stator_tooth', 'stator_winding']